In [2]:
# Seeing if PySpark works

import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.14540416


In [ ]:
import json
import sys, os
import argparse
import numpy as np
import findspark
findspark.init()
import pyspark
import random
import pandas as pd

#from multiprocessing import Pool

# Getting config
with open('/dfs/scratch1/jdunnmon/data/memex-data/config/config_spark.json') as fl:
    config = json.load(fl)

# Adding path for utils
sys.path.append('/dfs/scratch1/jdunnmon/repos/extractors/src/utils')

# Setting random seed
seed = config['seed']
random.seed(seed)
np.random.seed(seed)

# If memex_raw_content is a content_field, uses term as a regex in raw data in addition to getting title and body
#term = r'\b[Ll]ocation:|\b[cC]ity:'
#term = r'([Ll]ocation:.{0,100}|[cC]ity:.{0,100}|\d\dyo\W|\d\d.{0,10}\Wyo\W|\d\d.{0,10}\Wold\W|\d\d.{0,10}\Wyoung\W|\Wage\W.{0,10}\d\d)'
term = r'([Ll]ocation:[\w\W]{1,200}</.{0,20}>|\W[cC]ity:[\w\W]{1,200}</.{0,20}>|\d\dyo\W|\d\d.{0,10}\Wyo\W|\d\d.{0,10}\Wold\W|\d\d.{0,10}\Wyoung\W|\Wage\W.{0,10}\d\d)'

# Setting up arguments (can also get from argparse)
args = {}
args['data_loc'] = '/dfs/scratch1/jdunnmon/data/memex-data/tsvs/output_all_slicetest'
# Getting raw_content column
print('Getting correct column...')
files = os.listdir(args['data_loc'])
df = pd.read_csv(os.path.join(args['data_loc'],files[0]),sep='\t',nrows=10)
col = df.columns.get_loc("memex_raw_content")

# Getting path and setting up data
path = args['data_loc']
file_list = os.listdir(path)
path_list = [os.path.join(path, file) for file in file_list]
file_data = [(path, term, col) for path in path_list if path.endswith('tsv')]

out_dir = path + '/parsed/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

print('Processing in parallel')


sc = pyspark.SparkContext(appName="parse_html")
sc.addPyFile('/dfs/scratch1/jdunnmon/repos/extractors/src/utils/dataset_utils.py')
from dataset_utils import parse_html
print('Distributing data...')
distData = sc.parallelize(file_data)
print('Parsing distributed data...')
distData.foreach(parse_html)
sc.stop()
#threads = 30
#pool = Pool(threads) 
#results = pool.map(parse_html, file_data)
#pool.close()
#pool.join()


Getting correct column...
Processing in parallel
Distributing data...
Parsing distributed data...


In [5]:
sc.stop()